# PyMongo

Start the MongoDB server before running
- On Windows, click on the application to run the server
- On Mac, run ```brew services start mongodb-community@4.2``` (depending on what version you have downloaded)

In [22]:
import pymongo

# create client connection to database
client = pymongo.MongoClient("localhost", 27017)
# client = pymongo.MongoClient('127.0.0.1', 27017)

# create database
library = client["library"]

# create collection
books = library["books"]

# insert document
# document = { "_id":1, "book_title":"Tom's adventures"}
# document = { "_id":2, "book_title":"Mary's happy day"}
# x = collection.insert_one(document)
# print(x.inserted_id)

# query documents
results = books.find({})
print(results) # returns a cursor object
for result in results:
    print(result) # empty database currently

# close client connection
client.close()

# NoSQL
- Optional fields
- Different field types

## Load Data into Database

### Define schema (dynamic, so it can be changed)
```
{"uid": STRING,
"book_title" STRING,
"subject": STRING,
"summary": STRING,
"author_name": STRING,
"published": STRING,
"language": STRING}
```

In [23]:
import csv

data_link = "Datasets/data.gov.sg/national-library-board-read-singapore-short-stories/national-library-board-read-short-story.csv"

with open (data_link, "r") as data_file:
    # csv allows easy reading of data files with defined delimiter
    file_data = csv.reader(data_file, delimiter=",") # returns a csv object
    file_data = [data for data in file_data]

# print(file_data[0])
# fields:
# ['uid', 'book_title', 'subject', 'summary', 'original_publisher', 
# 'digital_publisher', 'format', 'language', 'copyright', 'author_name', 
# 'published', 'resource_url', 'cover', 'thumbnail']

books_data = [] # store all data as an array of dictionaries
for i in range (1, len(file_data)): # skip initial header row
    data = file_data[i]
    books_data.append({
        "_id": data[0],
        "book_title": data[1],
        "subject": data[2],
        "summary": data[3],
        "author_name": data[9],
        "published": int(data[10]),
        "language": data[7]
    })

print(books_data[0])

{'_id': 'c4489a23-e67b-4e1b-8864-029a22ae8f08', 'book_title': 'Air Sebelanga', 'subject': 'Under One Sky', 'summary': 'Apabila Nalini hanya mampu mendapatkan sebelanga air untuk hari itu, dia bimbang akan leteran ibu mentuanya. Sedang dia sibuk menyelesaikan masalahnya, tidak ramai yang sedia membantu.', 'author_name': 'Fehmida Zakeer', 'published': 2013, 'language': 'may'}


# CRUD: Create, Read, Update, Delete

### C - create/insert
### R - read/select/query
### U - update/edit
### D - delete/remove

## CREATE (INSERT)

In [24]:
import pymongo

# create client connection to database
client = pymongo.MongoClient("localhost", 27017)
# client = pymongo.MongoClient('127.0.0.1', 27017)

# get database
library = client["library"]

# get collection
books = library["books"]

# load data into collection
books.insert_many(books_data)
print("Inserted successfully!")

print("-" * 50)

# query documents
results = books.find({}).limit(2) # limit(n) returns first n items
print(results) # returns a cursor object
for result in results:
    print(result) # empty database currently

# close client connection
client.close()

Inserted successfully!
--------------------------------------------------
{'_id': 'c4489a23-e67b-4e1b-8864-029a22ae8f08', 'book_title': 'Air Sebelanga', 'subject': 'Under One Sky', 'summary': 'Apabila Nalini hanya mampu mendapatkan sebelanga air untuk hari itu, dia bimbang akan leteran ibu mentuanya. Sedang dia sibuk menyelesaikan masalahnya, tidak ramai yang sedia membantu.', 'author_name': 'Fehmida Zakeer', 'published': 2013, 'language': 'may'}
{'_id': '4793b784-868f-4238-9a1a-58698f27f46d', 'book_title': 'A Mother Like Her', 'subject': 'Roads Less Travelled', 'summary': 'A couple anticipated the birth of their first child and hoped that they could raise him to be a model citizen. However the father, the sole breadwinner of the family, passed away suddenly. The burden was placed on the young uneducated widow to raise her child alone. Determined to fulfill her husband’s dream, she worked day and night. Her continued absence from home had an adverse effect on her son. He took a deadly 

## READ (QUERY)

In [37]:
# NoSQL
# - optional fields
# - different field types

import pymongo
import json

# load / loads
# dump / dumps

# create connection
client = pymongo.MongoClient('localhost', 27017)

# create database
library = client["library"]

# create collection
books = library["books"] # equivalent to table in SQL

print("Querying all data...")
query = {}
fields = {"book_title": 1, "language": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).limit(2)
for result in results:
    print(result)
print("-" * 50)

print("Sorting all data by year published...")
query = {}
fields = {"book_title": 1, 
          "language": 1,
         "published": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).sort("published", pymongo.ASCENDING).limit(5) # sort in ascending order
for result in results:
    print(result)
print("-" * 50)

print("Getting all data where year published >= 2013...")
query = {"published": {"$gte": 2013}} # gte (>=), gt (>), lte (<=), lt (<)
fields = {"book_title": 1, 
          "language": 1,
         "published": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).limit(2)
for result in results:
    print(result)
print("-" * 50)

print("Getting all data where length of book_title <= 10...")
length = 10
query = {"$where": f"this.book_title.length <= {length}"} # use f-string for easy formatting
fields = {"book_title": 1, 
          "language": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).limit(5)
for result in results:
    print(result)
print("-" * 50)

print("Getting all data where length of book_title <= 10 AND language is English...")
length = 10
query = {"$and": [{"$where": f"this.book_title.length <= {length}"}, {"language": "eng"}]}
fields = {"book_title": 1, 
          "language": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).limit(5)
for result in results:
    print(result)
print("-" * 50)

print("Getting all data where subject is 'Under One Sky' OR language is Malay...")
length = 10
query = {"$or": [{"subject": "Under One Sky"}, {"language": "may"}]}
fields = {"book_title": 1, 
          "language": 1,
         "subject": 1} # fields that we want to return (leave it empty to return all fields)
results = books.find(query, fields).limit(5)
for result in results:
    print(result)
print("-" * 50)


# close connection
client.close()

Querying all data...
{'_id': 'c4489a23-e67b-4e1b-8864-029a22ae8f08', 'book_title': 'Air Sebelanga', 'language': 'may'}
{'_id': '4793b784-868f-4238-9a1a-58698f27f46d', 'book_title': 'A Mother Like Her', 'language': 'eng'}
--------------------------------------------------
Sorting all data by year published...
{'_id': 'fbd91bae-3852-4dbb-91a3-c0edb27f23f4', 'book_title': 'Gypsies', 'published': 2007, 'language': 'eng'}
{'_id': 'c64eb6ea-1243-41d3-b855-3731620797b9', 'book_title': 'Lee', 'published': 2007, 'language': 'eng'}
{'_id': 'b4a0ccbf-eb1f-4088-a6d7-4d3a169ffa55', 'book_title': 'Morning Dew', 'published': 2007, 'language': 'eng'}
{'_id': '9c657242-8467-4820-9a61-9bf4eaecd20f', 'book_title': 'Tanjong Rhu', 'published': 2007, 'language': 'eng'}
{'_id': '5e4e02be-c565-47f4-9112-60d070a25c77', 'book_title': 'Teenagers', 'published': 2007, 'language': 'eng'}
--------------------------------------------------
Getting all data where year published >= 2013...
{'_id': 'c4489a23-e67b-4e1b-8

## UPDATE (SET)

In [40]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient("localhost", 27017)

# get database
library = client["library"] 

# get collection
books = library["books"]

# insert into collection

# {"uid": STRING,
# "book_title" STRING,
# "subject": STRING,
# "summary": STRING,
# "author_name": STRING,
# "published": STRING,
# "language": STRING}

# automatically assigned unique _id if not set

# dynamic schema and data type
documents = [
    {"_id": 234, "book_title": "Father Night", "author_name": "Kurt", 
     "publisher": "APress", "page_count": 433, "published": "2018"},
    {"_id": 134, "book_title": "Mother Night", "author_name": ["Kurt", "Dan"], 
     "publisher": "APress", "published": "2015"},
    {"_id": 334, "book_title": "Programming C## 6.0", "author_name": ["Andrew", "Dan"], 
     "page_count": 300, "published": "2000"},
    {"_id": 534, "book_title": "Introduction to Python", 
     "publisher": "MPH", "published": "1999"},
    {"_id": 434, "book_title": "Travel with Dogs", "author_name": "Andy", 
     "publisher": "APress", "page_count": 100, "published": "2017"}
]             
books.insert_many(documents)
print("Inserted successfully!")

# close connection
client.close()

Inserted successfully!


In [48]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient("localhost", 27017)

# get database
library = client["library"] 

# get collection
books = library["books"]


print("Get title, author for books published in 2015")
criteria = {"published": "2015"}
docs = books.find(criteria, {"book_title": 1, "author_name": 1})
for doc in docs:
    print(doc)
print("-" * 50)

print("Get books where 100 <= page_count < 400")
criteria = {"$and": [{"page_count": { "$gte": 100 }}, {"page_count": { "$lt": 400 }}]}
# criteria = {"page_count": {"$gte":100, "$lt":400}}
docs = books.find(criteria)
for doc in docs:
    print(doc)   
print("-" * 50)

print("Update page_count for documents without page_count to 'Less Than 100 Pages'")
criteria = {"page_count": {"$exists": False}}
new_value = {"$set": {"page_count": "Less Than 100 Pages"}}
books.update_many(criteria, new_value)
docs = books.find().limit(5)
for doc in docs:
    print(doc)
print("-" * 50)


# close connection
client.close()

Get title, author for books published in 2015
{'_id': 134, 'book_title': 'Mother Night', 'author_name': ['Kurt', 'Dan']}
--------------------------------------------------
Get books where 100 <= page_count < 400
{'_id': 334, 'book_title': 'Programming C## 6.0', 'author_name': ['Andrew', 'Dan'], 'page_count': 300, 'published': '2000'}
{'_id': 434, 'book_title': 'Travel with Dogs', 'author_name': 'Andy', 'publisher': 'APress', 'page_count': 100, 'published': '2017'}
--------------------------------------------------
Update page_count for documents without page_count to 'Less Than 100 Pages'
{'_id': 'c4489a23-e67b-4e1b-8864-029a22ae8f08', 'book_title': 'Air Sebelanga', 'subject': 'Under One Sky', 'summary': 'Apabila Nalini hanya mampu mendapatkan sebelanga air untuk hari itu, dia bimbang akan leteran ibu mentuanya. Sedang dia sibuk menyelesaikan masalahnya, tidak ramai yang sedia membantu.', 'author_name': 'Fehmida Zakeer', 'published': 2013, 'language': 'may', 'page_count': 'Less Than 10

## DELETE

In [51]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient("localhost", 27017)

# get database
library = client["library"] 

# get collection
books = library["books"]

print("Delete documents with 'Less Than 100 Pages'")
criteria = {"page_count": "Less Than 100 Pages"}
initial_count = books.count_documents(criteria)
books.delete_one(criteria) # delete one
final_count = books.count_documents(criteria)
docs = books.find(criteria)
print(f"There were initially {initial_count} documents, but now there are {final_count} documents matching the criteria.")
print("-" * 50)

# close connection
client.close()

Delete documents with 'Less Than 100 Pages'
There were initially 120 documents, but now there are 119 documents matching the criteria.
--------------------------------------------------


In [52]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient("localhost", 27017)

# get database
library = client["library"] 

# get collection
books = library["books"]

print("Delete all documents with subject 'Under One Sky'")
criteria = {"subject": "Under One Sky"}
initial_count = books.count_documents(criteria)
books.delete_many(criteria) # delete all matching
final_count = books.count_documents(criteria)
docs = books.find(criteria)
print(f"There were initially {initial_count} documents, but now there are {final_count} documents matching the criteria.")
print("-" * 50)

# close connection
client.close()

Delete all documents with subject 'Under One Sky'
There were initially 59 documents, but now there are 0 documents matching the criteria.
--------------------------------------------------


In [53]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient("localhost", 27017)

# get database
library = client["library"] 

# get collection
books = library["books"]

# delete colleciton
library.drop_collection(books)
print("Removed books collection!")

client.drop_database(library)
print("Removed library database!")

# close connection
client.close()

Removed books collection!
Removed library database!
